In [1]:
import pandas as pd
import numpy as np

'''Found this pretty good implementation of apriori'''
from mlxtend.frequent_patterns import apriori
from IPython.display import clear_output

In [2]:
divider = 5                     # Divide every m/z by this number.
max_len = int(10000 / divider)  # Max allowed m/z value.
filter_peaks = 100              # How many peaks to keep after filteration.
max_itemset_size = 7            # Max size of the frequent itemsets.
min_support = 0.05              # Maybe we should increase this?

In [3]:
'''To keep updating this dict for multiple files. Dont define it again.'''
pep_spec = {}

#### Magic reader  
You just need to provide the file path and it'll do everything for you. :)

In [4]:
f=open("human_consensus_final_true_lib.msp", "r")
lines = f.readlines()
f.close()

isName = isMW = isNumPeaks = False
new = prev = 0
i = 0


while i < len(lines):
    '''
    Process each line of the file.
    '''
    line = lines[i]
    i += 1
    
    splits = line.split(':')
    
    '''Keep going through lines. Name line is peptide and
    all the lines after Num peaks are the spectrum'''
    if splits[0] == 'Name':
        split1 = splits[1]
        pep = split1.split('/')[0].lstrip(' ')
        isName = True
        
    if isName and splits[0] == 'MW':
        mass = float(splits[1]) # Not using it anywhere. It's just there.
        isMW = True
        
    if isName and isMW and splits[0] == 'Num peaks':
        '''After this line, we will start processing the spectrum.'''
        numPeaks = int(splits[1]) # Just in case.
        
        temp_spec = np.zeros(max_len)
        while (lines[i] != '\n'):
            mzline = lines[i]
            i += 1
            mzsplits = mzline.split('\t')
            # Get the m/z and intenisty value of each line.
            moz, intensity = float(mzsplits[0]), float(mzsplits[1])
            # Dividing m/z by the divider.
            temp_spec[round(moz / divider)] = intensity
        
        isNumPeaks = True
        
    if isName and isMW and isNumPeaks:
        '''At this point, we are done reading one spectrum.
        Place everything where it needs to go and read the next one.'''
        isName = isMW = isNumPeaks = False
        
        # This is the filter. Gets the top 100 peaks
        # and returns them in one hot encoded vector of size max_len
        top_indx = np.argpartition(temp_spec, -filter_peaks)[-filter_peaks:]
        spec = np.zeros(max_len)
        spec[top_indx] = 1
        
        # Creating dictionary. Key: peptide, Value: spectrum.
        # As a side effect we'll get rid of duplicates.
        if pep not in pep_spec: pep_spec[pep] = spec
        
        # Displays the progress.
        new = int((i/len(lines)) * 100)
        if new > prev:
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new


99%


In [5]:
# Just get the value from the dict.
buckets = list(pep_spec.values())
print(len(buckets))

207910


In [6]:
# The apriori function needs a dataframe.
df_buckets = pd.DataFrame(buckets[0:50000], columns=list(range(0, max_len)))

#### Documentation of apriori  
http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

In [ ]:
freq_itemsets = apriori(df_buckets, 
                        min_support=min_support, 
                        max_len=max_itemset_size, 
                        verbose=1, 
                        low_memory=True)